1. Grid Search
2. Try LogReg with All Parameters


In [1]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline



In [2]:
data_path_raw = Path.cwd().parent.parent / "data" / "raw"
data_path_preprocessed = Path.cwd().parent.parent / "data" / "processed"

In [3]:
raw_df_processed = pd.read_csv(data_path_preprocessed / "training_set_features_encoded_imputed_standardized.csv", index_col="respondent_id")
labels_df = pd.read_csv(data_path_raw / "training_set_labels.csv", index_col="respondent_id")
test_df_processed = pd.read_csv(data_path_preprocessed / "test_set_features_encoded_imputed_standardized.csv", index_col="respondent_id")

all_raw_df = raw_df_processed.join(labels_df)

In [4]:
import time 

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [5]:
#3 
X_train, X_test, y_train, y_test = train_test_split(
        raw_df_processed,
        labels_df,
        shuffle = True,
        test_size = 0.25,
        random_state = 10)


In [6]:
pipeline = Pipeline([
            ('LogReg', MultiOutputClassifier(LogisticRegression())) 
                 ])

In [7]:
from sklearn.model_selection import GridSearchCV

param = {
    'LogReg__estimator__penalty' : ['l1', 'l2' ,'elasticnet', 'none'],
    'LogReg__estimator__dual' : [False, True],
    'LogReg__estimator__C' : [0.5,1.,2.,5.],
    'LogReg__estimator__fit_intercept' : [False, True], 
    'LogReg__estimator__class_weight' :['balanced', 'None'],
    'LogReg__estimator__solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'LogReg__estimator__max_iter' : [100],
    'LogReg__estimator__l1_ratio' : [0, 0.5, 1]
    }    

clf = GridSearchCV(estimator=pipeline, param_grid = param, cv = 5, n_jobs=-1)
clf.fit(raw_df_processed, labels_df)




/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('LogReg',
                                        MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                                           class_weight=None,
                                                                                           dual=False,
                                                                                           fit_intercept=True,
                                                                                           intercept_scaling=1,
                                                                                           l1_ratio=None,
                                                                                           max_iter=100,
                                                                                           multi_class='auto',
             

In [8]:
clf.best_params_


{'LogReg__estimator__C': 1.0,
 'LogReg__estimator__class_weight': 'None',
 'LogReg__estimator__dual': False,
 'LogReg__estimator__fit_intercept': True,
 'LogReg__estimator__l1_ratio': 0,
 'LogReg__estimator__max_iter': 100,
 'LogReg__estimator__penalty': 'elasticnet',
 'LogReg__estimator__solver': 'saga'}

{'LogReg__estimator__C': 1.0,
 'LogReg__estimator__class_weight': 'None',
 'LogReg__estimator__dual': False,
 'LogReg__estimator__fit_intercept': True,
 'LogReg__estimator__l1_ratio': 0,
 'LogReg__estimator__max_iter': 100,
 'LogReg__estimator__penalty': 'elasticnet',
 'LogReg__estimator__solver': 'saga'}

In [9]:
results = pd.DataFrame(clf.cv_results_)
results[results['rank_test_score'] < 10]
top_results = results[results['rank_test_score'] < 10].sort_values(ascending = True, by = 'rank_test_score')
top_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_LogReg__estimator__C,param_LogReg__estimator__class_weight,param_LogReg__estimator__dual,param_LogReg__estimator__fit_intercept,param_LogReg__estimator__l1_ratio,param_LogReg__estimator__max_iter,...,param_LogReg__estimator__solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
794,8.357800,0.285498,0.009268,0.002596,1,None,False,True,0,100,...,saga,"{'LogReg__estimator__C': 1.0, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.677027,0.667104,0.667353,0.005098,1
309,7.470716,0.061710,0.006347,0.000586,0.5,None,False,True,0,100,...,saga,"{'LogReg__estimator__C': 0.5, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.676840,0.667104,0.667316,0.005027,2
314,7.852949,0.299068,0.008382,0.001623,0.5,None,False,True,0,100,...,saga,"{'LogReg__estimator__C': 0.5, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.676840,0.667104,0.667316,0.005027,2
329,8.074111,0.041862,0.010962,0.003018,0.5,None,False,True,0.5,100,...,saga,"{'LogReg__estimator__C': 0.5, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.676840,0.667104,0.667316,0.005027,2
349,8.406183,0.481477,0.009301,0.004344,0.5,None,False,True,1,100,...,saga,"{'LogReg__estimator__C': 0.5, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.676840,0.667104,0.667316,0.005027,2
789,7.919044,0.156186,0.007225,0.002085,1,None,False,True,0,100,...,saga,"{'LogReg__estimator__C': 1.0, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.677027,0.666916,0.667316,0.005100,2
809,7.251581,0.094205,0.006528,0.000959,1,None,False,True,0.5,100,...,saga,"{'LogReg__estimator__C': 1.0, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.677027,0.666916,0.667316,0.005100,2
829,7.986551,0.142872,0.008420,0.002572,1,None,False,True,1,100,...,saga,"{'LogReg__estimator__C': 1.0, 'LogReg__estimat...",0.662112,0.665481,0.665044,0.677027,0.666916,0.667316,0.005100,2
1779,7.795985,0.085977,0.006433,0.000643,5,None,False,True,0.5,100,...,saga,"{'LogReg__estimator__C': 5.0, 'LogReg__estimat...",0.662112,0.665668,0.665044,0.677027,0.666729,0.667316,0.005091,9
1769,7.722016,0.251085,0.006200,0.000440,5,None,False,True,0.5,100,...,saga,"{'LogReg__estimator__C': 5.0, 'LogReg__estimat...",0.662112,0.665668,0.665044,0.677027,0.666729,0.667316,0.005091,9


In [10]:
#clf.cv_results_['params'][404]

In [11]:
logisticRegr = MultiOutputClassifier(LogisticRegression(
                        C = 0.5,
                        class_weight = None,
                        dual = False,
                        fit_intercept = True,
                        max_iter = 1000,
                        penalty = 'l1', 
                        solver = 'saga'))

logisticRegr.fit(X_train, y_train)
test_probability = logisticRegr.predict_proba(X_test)


In [12]:
y_preds = pd.DataFrame(
{
        "h1n1_vaccine": test_probability[0][:, 1],
        "seasonal_vaccine": test_probability[1][:, 1],
    },
    index = y_test.index
    )

In [13]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_preds))

0.8406943468907931


In [20]:
logisticRegr = MultiOutputClassifier(LogisticRegression(
                        C = 1.0,
                        class_weight = None,
                        dual = False,
                        fit_intercept = True,
                        l1_ratio = 0,
                        max_iter = 100,
                        penalty = 'elasticnet', 
                        solver = 'saga'))

logisticRegr.fit(raw_df_processed, labels_df)
test_probability = logisticRegr.predict_proba(test_df_processed)

/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/Users/johannes/opt/anaconda3/envs/py37-ds/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [21]:
predictions = pd.DataFrame(
    {
        "h1n1_vaccine": test_probability[0][:, 1],
        "seasonal_vaccine": test_probability[1][:, 1],
    },
    index = test_df_processed.index
)


In [22]:
submission_df = pd.read_csv(data_path_raw / "submission_format.csv", 
                            index_col="respondent_id")

In [23]:
np.testing.assert_array_equal(test_df_processed.index.values, 
                              submission_df.index.values)


In [24]:
# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = predictions["h1n1_vaccine"]
submission_df["seasonal_vaccine"] = predictions["seasonal_vaccine"]

In [25]:
#CHANGE PATH 

output_path = Path.cwd().parent.parent / "models" / "submissions"


submission_df.to_csv(output_path /'logreg_JFR_200729_GridSearch_HyperparameterTuning.csv', index=True)

